In [56]:
import numpy as np
import polars as pl
from ortools.sat.python import cp_model

armors = pl.read_parquet("./data/armor_pieces.parquet")

In [57]:
solver_statuses = {
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.UNKNOWN: "UNKNOWN",
}

In [58]:
# var = model.NewIntVar(lb=0, ub=10, name="my_var1")

# model.maximize(var)

# solver = cp_model.CpSolver()
# status = solver.solve(model)

# solver_statuses[status]

# solver.value(var)

In [59]:
stuffs = [f"piece_de_stuff_{i}" for i in range(10)]

_vars = {"is_stuff_equipped": {}}

model = cp_model.CpModel()

for stuff in stuffs:
    is_equipped = model.NewBoolVar(f"is_equipped_{stuff}")
    _vars["is_stuff_equipped"][stuff] = is_equipped
model.Add(sum(_vars["is_stuff_equipped"][stuff] for stuff in stuffs) <= 1)

In [60]:
armors = armors.select("piece", "name", "talent_name", "talent_level")
armors

piece,name,talent_name,talent_level
str,str,str,i64
"""Tête""","""Masque d'espoir""","""Crâne d'acier""",1
"""Torse""","""Cotte d'espoir""","""Bénédiction""",1
"""Bras""","""Avant-bras d'espoir""","""Bénédiction""",1
"""Taille""","""Tassette d'espoir""","""Crâne d'acier""",1
"""Jambes""","""Grèves d'espoir""","""Bénédiction""",1
…,…,…,…
"""Tête""","""Mimiflore α""","""Pelage de renforcement""",1
"""Tête""","""Mimiflore α""","""Entomologiste""",1
"""Tête""","""Mimiflore α""","""Embuscade""",1


In [61]:
unique_gear_types = armors["piece"].unique().sort().to_list()

_vars = {
    "is_piece_equipped": {gear_type: {} for gear_type in unique_gear_types},
}


unique_gear_type = unique_gear_types[0]
sub_df = armors.filter(pl.col("piece") == unique_gear_type)

piece_names = sub_df["name"].unique().sort().to_list()
piece_name = piece_names[0]

for row in sub_df.filter(pl.col("name") == piece_name).iter_rows():
    break
piece_type, piece_name, talent, talent_lvl = row

In [62]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

In [63]:
my_var = model.NewIntVar(lb=0, ub=100, name="my_var")

In [64]:
model.Add(my_var <= 50)

In [65]:
model.maximize(obj=my_var)

In [66]:
solver_statuses = {
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.OPTIMAL: "OPTIMAL",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.UNKNOWN: "UNKNOWN",
}

# Instantiate solver
solver = cp_model.CpSolver()
# Solve
status = solver.solve(model)
print(f"Solver solution: {solver_statuses[status]}")

print(f"Value of my_var: {solver.value(my_var)}")

Solver solution: OPTIMAL
Value of my_var: 50


In [67]:
# Load charm data
charms = pl.read_parquet("./data/charms.parquet")
# Get unique charm names
charm_names = charms["name"].unique().sort().to_list()

# Instantiate the CpModel
model = cp_model.CpModel()

# Instantiate the variables
vars_charm_equipped = {}

# Create one boolean variable telling if the charm
# is equipped or not for each charm
for charm_name in charm_names:
    vars_charm_equipped[charm_name] = model.NewBoolVar(
        name=f"charm_equipped_{charm_name}"
    )

# Add the constraint that only one charm can be equipped at a time
model.Add(sum(vars_charm_equipped.values()) <= 1)

# Define a bull*** objective to tell the model to use
# the first alphabetical charm
model.maximize(vars_charm_equipped[charm_names[0]])

# Solve the model
status = solver.solve(model)

# Display the solution
for charm_name, var in vars_charm_equipped.items():
    if solver.Value(var) == 1:
        print(f"Charm {charm_name} is equipped")

Charm Talisman anti-immobilisation is equipped


In [68]:
# Load charm data
charms = pl.read_parquet("./data/charms.parquet")
# Get unique charm names
charm_names = charms["name"].unique().sort().to_list()
# Gather skill names
unique_skills = charms["talent_name"].unique().sort().to_list()

# Instantiate a variable registry to store variables
var_registry = {
    "charm_equipped": {},
    "skill_list": {skill: [] for skill in unique_skills},
    "skill_sum": {},
}


# Instantiate the CpModel
model = cp_model.CpModel()

# For each charm
for charm_name in charm_names:
    # Get data from the specific charm
    filtered_charm = charms.filter(pl.col("name") == charm_name)

    # Create a variable indicating that the
    # charm is equipped or not
    is_charm_equipped = model.NewBoolVar(f"charm_{charm_name}_equipped")

    # As a charm may have multiple skills,
    # we iterate over each skill
    for row in filtered_charm.to_dicts():
        charm_name, skill_name, skill_lvl = (
            row[k] for k in ["name", "talent_name", "talent_lvl"]
        )

        # Instantiate a variable that indicates that
        # equiping the charm will give us skill points
        skill_points_inherited = model.NewIntVar(
            lb=0,
            ub=100,
            name=f"skill_{skill_name}_points_inherited_from_{charm_name}",
        )

        # Force the amount of skill points to be
        # the skill points the charm has,
        # but only if the charm is equipped
        model.Add(skill_points_inherited == skill_lvl).only_enforce_if(
            is_charm_equipped
        )

        # If the charm is not equipped,
        # force the amount of skill points to be 0
        model.Add(skill_points_inherited == 0).only_enforce_if(is_charm_equipped.Not())

        # Register the amount of skill points variable
        # into the registry
        var_registry["skill_list"][skill_name].append(skill_points_inherited)

    # Register the fact that the charm is equipped or
    # not into the registry
    var_registry["charm_equipped"][charm_name] = is_charm_equipped

# Add the constraint to have at maximum one charm equipped
model.Add(sum(var_registry["charm_equipped"].values()) <= 1)

# For each skill, we may have multiple potential charms
# For this reason, we sum the total of skill points
# for each skill into one single variable
for skill_name, var_list in var_registry["skill_list"].items():
    # Create an integer variable that is the sum of skill
    # points for a specific skill
    skill_sum = model.NewIntVar(
        lb=0,
        ub=100,
        name=f"skill_sum_for_{skill_name}",
    )
    # Force this variable to be the sum of skill
    # points inherited from all charms for this
    # specific skill
    model.Add(skill_sum == sum(var_list))
    var_registry["skill_sum"][skill_name] = skill_sum


# Create an objective to maximize a specific skill (here "Botaniste")
model.maximize(obj=var_registry["skill_sum"]["Botaniste"])

# Solve the model
status = solver.solve(model)

# Display the solution
for charm_name, var in var_registry["charm_equipped"].items():
    if solver.Value(var) == 1:
        charm_equipped = charm_name

solution = charms.filter(pl.col("name") == charm_equipped)
for row in solution.to_dicts():
    print(
        f"Charm: {row['name']}\nSkill: {row['talent_name']}\nSkill level: {row['talent_lvl']}"
    )

Charm: Talisman de botanique IV
Skill: Botaniste
Skill level: 4


In [69]:
# Load the skills dataset
skills = pl.read_parquet("./data/talents.parquet")

# Instantiate skill sum capped in the registry
var_registry["skill_sum_capped"] = {}

# For each skill
for skill_name, var_skill_sum in var_registry["skill_sum"].items():
    # Get the max skill level from the dataset
    max_level = (
        skills.filter(pl.col("name") == skill_name)
        .explode("levels")
        .select(pl.col("levels").struct.field("lvl").max().alias("lvl"))
        .to_dicts()[0]["lvl"]
    )

    # Create the new proxy variable
    var_skill_capped = model.NewIntVar(
        lb=0, ub=100, name=f"skill_sum_capped_for_skill_{skill_name}"
    )

    # Constraint the new proxy variable to be the skill sum
    # but which can't go past the maximum skill level
    model.AddMinEquality(
        target=var_skill_capped,
        exprs=[
            var_registry["skill_sum"][skill_name],
            max_level,
        ],
    )

    # Register the new proxy variable
    var_registry["skill_sum_capped"][skill_name] = var_skill_capped

In [70]:
# Create new proxy variable to store the level of group skills
var_registry["group_skill_sum_capped"] = {}

# Filter only group skills
group_skills = skills.filter(pl.col("group") == "Group")

# Get unique group skill names
skill_names = group_skills["name"].unique().sort().to_list()

# For each group skill existing in the registry
for skill_name in set(skill_names).intersection(
    var_registry["skill_sum_capped"].keys()
):
    # Get maximum level required for this group skill
    group_skill_level = (
        group_skills.filter(pl.col("name") == skill_name)
        .select(
            pl.col("levels")
            .explode()
            .struct.field("lvl")
            .max()
            .alias("group_skill_level")
        )
        .to_dicts()[0]["group_skill_level"]
    )

    # Create boolean variable to track if group has enough skill levels
    var_group_has_enough_levels = model.NewBoolVar(
        f"group_skill_{skill_name}_has_enough_levels"
    )
    # If the group has enough levels,
    # then we set the boolean variable to true
    model.Add(
        var_registry["skill_sum_capped"][skill_name] >= group_skill_level
    ).OnlyEnforceIf(var_group_has_enough_levels)
    # If the group does not have enough levels,
    # then we set the boolean variable to false
    model.Add(
        var_registry["skill_sum_capped"][skill_name] < group_skill_level
    ).OnlyEnforceIf(var_group_has_enough_levels.Not())

    # We create a new proxy variable
    # that will be the capped group skill sum
    var_group_skill_sum_capped = model.NewIntVar(
        lb=0,
        ub=100,
        name=f"group_skill_sum_capped_{skill_name}",
    )

    # Set capped sum equal to actual sum if enough levels
    model.Add(
        var_group_skill_sum_capped == var_registry["skill_sum_capped"][skill_name]
    ).OnlyEnforceIf(var_group_has_enough_levels)
    # Set capped sum to 0 if not enough levels
    model.Add(var_group_skill_sum_capped == 0).OnlyEnforceIf(
        var_group_has_enough_levels.Not()
    )

    # Register the variable into the registry
    var_registry["group_skill_sum_capped"][skill_name] = var_group_skill_sum_capped

In [71]:
from itertools import pairwise

# Instantiate new proxy variables for series skills
var_registry["series_skill_sum_capped"] = {}

# Get unique series skills names
series_skill_names = (
    skills
    #
    .filter(pl.col("group") == "Series")["name"]
    .unique()
    .sort()
    .to_list()
)

# For each series skills that exist in the registry
for skill_name in set(series_skill_names).intersection(
    var_registry["skill_sum_capped"].keys()
):
    # Get the list of level thresholds
    skill_levels = (
        skills.filter(pl.col("name") == skill_name)
        .select(pl.col("levels").explode().struct.field("lvl"))["lvl"]
        .sort()
        .to_list()
    )
    # Add extremum values for easier pairwise manipulation
    skill_levels = [0] + skill_levels + [100]

    # Instantiate a list that will store, for this specific skill,
    # individual variables that represents the fact that
    # the skill levels are between lower and upper threshold in the following way:
    # if between the threshold, the value is the lower threshold
    # if not, the value is 0.
    # The final skill level will be the sum of those individual variables
    interval_variables = []
    # Iterate ofver pairwise level threhsolds
    for skill_level_inferior, skill_level_superior in pairwise(skill_levels):
        # Create a boolean variable to check if the skill level is more than the inferior level
        var_skill_lvl_is_more_than_inferior = model.NewBoolVar(
            name=f"skill_series_{skill_name}_greater_than_{skill_level_inferior}"
        )
        # Define the strict constraints to bind the boolean variable to
        # the actual fact that the skill level is more than the inferior level
        model.Add(
            skill_level_inferior <= var_registry["skill_sum_capped"][skill_name]
        ).only_enforce_if(var_skill_lvl_is_more_than_inferior)
        model.Add(
            skill_level_inferior > var_registry["skill_sum_capped"][skill_name]
        ).only_enforce_if(var_skill_lvl_is_more_than_inferior.Not())

        # Create a boolean variable to check if the skill level is less than the superior level
        var_skill_lvl_is_less_than_superior = model.NewBoolVar(
            name=f"skill_series_{skill_name}_less_than_{skill_level_superior}"
        )
        # Define the strict constraints to bind the boolean variable to
        # the actual fact that the skill level is less than the superior level
        model.Add(
            skill_level_superior > var_registry["skill_sum_capped"][skill_name]
        ).only_enforce_if(var_skill_lvl_is_less_than_superior)
        model.Add(
            skill_level_superior <= var_registry["skill_sum_capped"][skill_name]
        ).only_enforce_if(var_skill_lvl_is_less_than_superior.Not())

        # Create a boolean variable to check if the skill level is between the inferior and superior levels
        var_skill_lvl_is_between = model.NewBoolVar(
            name=f"skill_series_{skill_name}_between_{skill_level_inferior}_and_{skill_level_superior}"
        )
        # Add constraints to enforce the 'between' condition
        model.Add(
            sum(
                [
                    var_skill_lvl_is_more_than_inferior,
                    var_skill_lvl_is_less_than_superior,
                ]
            )
            == 2
        ).only_enforce_if(var_skill_lvl_is_between)
        model.Add(
            sum(
                [
                    var_skill_lvl_is_more_than_inferior,
                    var_skill_lvl_is_less_than_superior,
                ]
            )
            < 2
        ).only_enforce_if(var_skill_lvl_is_between.Not())

        # Finally, if the skill level is between the inferior and superior levels,
        # we set the skill level to the inferior level
        skill_level = model.NewIntVar(
            name=f"skill_series_{skill_name}_level",
            lb=0,
            ub=100,
        )
        model.Add(skill_level == skill_level_inferior).only_enforce_if(
            var_skill_lvl_is_between
        )
        model.Add(skill_level == 0).only_enforce_if(var_skill_lvl_is_between.Not())
        interval_variables.append(skill_level)

    # Create the series skill proxy variables
    series_skill_sum = model.NewIntVar(
        lb=0,
        ub=100,
        name=f"series_skill_{skill_name}_sum",
    )
    # Set the series skill level to be the sum of the interval variables
    model.Add(series_skill_sum == sum(interval_variables))

    # Register the series skill proxy variable
    var_registry["series_skill_sum_capped"][skill_name] = series_skill_sum

In [75]:
var_registry["skill_sum_final"] = {}

# Add group skills to the final proxy variable
for skill_name, var in var_registry["group_skill_sum_capped"].items():
    var_registry["skill_sum_final"][skill_name] = var

# Add series skills to the final proxy variable
for skill_name, var in var_registry["series_skill_sum_capped"].items():
    var_registry["skill_sum_final"][skill_name] = var

# Add standard skills to the final proxy variable
skill_names = (
    set(var_registry["skill_sum_capped"].keys())
    - set(var_registry["group_skill_sum_capped"].keys())
    - set(var_registry["series_skill_sum_capped"].keys())
)
for skill_name in skill_names:
    var_registry["skill_sum_final"][skill_name] = var_registry["skill_sum_capped"][
        skill_name
    ]

In [77]:
total_amount_of_skill_points = sum(var_registry["skill_sum_final"].values())

In [73]:
wanted_skills = [
    {"name": "Botaniste", "weight": 2},
    {"name": "Un pour tous", "weight": 1},
]

skill_objective = sum(
    var_registry["skill_sum_final"][skill_name] * 10**skill_weight
    for skill_name, skill_weight in wanted_skills
)

# model.maximize(skill_objective)

TypeError: 'IntVar' object is not subscriptable

In [19]:
# Get unique gear types (head, torso, etc)
gear_types = armors["piece"].unique().sort().to_list()

# Instantiate a registry to store the fact that an armor piece is equipped or not
var_registry["armor_piece_equipped"] = {gear_type: {} for gear_type in gear_types}

# For each gear type
for gear_type in gear_types:
    # Get the subset of armors
    armors_filtered_gear_type = armors.filter(pl.col("piece") == gear_type)
    # Get the list of unique armor piece names
    armor_piece_names = armors_filtered_gear_type["name"].unique().sort().to_list()

    # Iterate over each individual armor piece in the game
    for armor_piece_name in armor_piece_names:
        # Get the data of the armor piece (skills & skill level)
        armor_data = armors_filtered_gear_type.filter(
            pl.col("name") == armor_piece_name
        ).to_dicts()

        # Create a boolean variable indicating the fact that
        # the armor piece is equipped or not
        armor_piece_name = armor_data[0]["name"]
        var_is_armor_piece_equipped = model.NewBoolVar(
            f"gear_type_{gear_type}_piece_name_{armor_piece_name}_is_equipped"
        )

        # Iterate on each skill that this specific armor piece provides
        for skill_data in armor_data:
            skill_name, skill_level = (
                skill_data[_] for _ in ["talent_name", "talent_level"]
            )

            # Create a variable that indicates that we benefit
            # a given amount of skill points from equipping
            # this specific armor piece
            var_skill_level_from_equipped_gear = model.NewIntVar(
                lb=0,
                ub=100,
                name=f"skill_points_for_skill_{skill_name}_gear_type_{gear_type}_piece_name_{armor_piece_name}",
            )
            # Add the constraints to force the variable to be equal to the skill level
            # if the armor piece is equipped, and 0 otherwise
            model.Add(
                var_skill_level_from_equipped_gear == skill_level
            ).only_enforce_if(var_is_armor_piece_equipped)
            model.Add(var_skill_level_from_equipped_gear == 0).only_enforce_if(
                var_is_armor_piece_equipped.Not()
            )

            # Store those skill points variables into the registry
            if skill_name not in var_registry["skill_list"].keys():
                var_registry["skill_list"][skill_name] = []

            var_registry["skill_list"][skill_name].append(
                var_skill_level_from_equipped_gear
            )

        # Store the fact that the armor piece is equipped or not
        var_registry["armor_piece_equipped"][gear_type][armor_piece_name] = (
            var_is_armor_piece_equipped
        )

# Create the constraints to specify that we can at most
# wear only one type of gear at a time
for gear_type in gear_types:
    model.Add(sum(var_registry["armor_piece_equipped"][gear_type].values()) <= 1)

In [20]:
armors = pl.read_parquet("data/armor_pieces.parquet")

armor_jewel_data = (
    armors
    #
    .select("piece", "name", *[f"jewel_{i}" for i in range(1, 4)]).unique()
)

print(armor_jewel_data.to_pandas().head(5).to_markdown(index=False))

| piece   | name                       |   jewel_1 |   jewel_2 |   jewel_3 |
|:--------|:---------------------------|----------:|----------:|----------:|
| Jambes  | Grèves Doshaguma Gardien   |         0 |         0 |         0 |
| Tête    | Heaume Doshaguma Gardien β |         1 |         2 |         0 |
| Bras    | Avant-bras Blangonga α     |         0 |         1 |         0 |
| Taille  | Tassette Nerscylla α       |         0 |         0 |         0 |
| Torse   | Cotte Gore Magala β        |         1 |         0 |         1 |


In [21]:
# We will store in the registry
# only variable for each armor piece and jewel type.
# Jewels of the same type/rank will be placed in the same list.
var_registry["jewel_emplacement_lists"] = {i: [] for i in range(1, 4)}
# We will create proxy variables that are the
# sum of available jewel emplacements for each
# jewel type/rank
var_registry["jewel_emplacement_sums"] = {}

# For each gear type (head, torso, etc)
for gear_type in gear_types:
    # Get the data subset
    armor_jewel_data_gear_filtered = armor_jewel_data.filter(
        pl.col("piece") == gear_type
    )

    # Iterate over each armor piece
    armor_names = armor_jewel_data_gear_filtered["name"].unique().to_list()
    for armor_name in armor_names:
        # Get the armor piece data
        armor_data = armor_jewel_data_gear_filtered.filter(
            pl.col("name") == armor_name
        ).to_dicts()[0]

        # For each jewel type
        for jewel_type in range(1, 4):
            # Retrieve the actual number of
            # jewel emplacement for this specific gear
            nb_emplacements_jewel = armor_data[f"jewel_{jewel_type}"]
            # Create a new variable that will store
            # the amount of jewels from this specific rank
            # inherited by equiping this specific piece
            # of gear
            var_nb_emplacements_jewel = model.NewIntVar(
                lb=0,
                ub=100,
                name=f"nb_emplacements_for_jewel_type_{jewel_type}_inherited_from_{armor_name}",
            )

            # Gather the boolean variable that indicates if we equipped
            # this specific piece or not
            is_armor_piece_equipped = var_registry["armor_piece_equipped"][gear_type][
                armor_name
            ]

            # Add the constraints: if the armor piece is equipped
            # the we consider that the number of jewel slots
            # are the ones from the data sheet
            model.Add(
                var_nb_emplacements_jewel == nb_emplacements_jewel
            ).only_enforce_if(is_armor_piece_equipped)
            # If the piece is not equipped, we consider
            # that the armor piece gives 0 jewel slots
            model.Add(var_nb_emplacements_jewel == 0).only_enforce_if(
                is_armor_piece_equipped.Not()
            )

            # Register the amount of jewels into the registry
            var_registry["jewel_emplacement_lists"][jewel_type].append(
                var_nb_emplacements_jewel
            )

# Aggregate the total number of available jewel slots into
# proxy variables
for jewel_type in range(1, 4):
    # Create a new proxy variable that will store the sum
    # of all jewel emplacement for this type/rank
    var_nb_of_jewel_emplacements = model.NewIntVar(
        lb=0,
        ub=100,
        name=f"total_nb_of_jewels_emplacements_of_type_{jewel_type}_from_equipped_gear",
    )
    # Create the aggregation rule via a strict constraint
    model.Add(
        var_nb_of_jewel_emplacements
        == sum(var_registry["jewel_emplacement_lists"][jewel_type])
    )
    # Register the sum in the variable registry
    var_registry["jewel_emplacement_sums"][jewel_type] = var_nb_of_jewel_emplacements

In [42]:
all_jewels = pl.read_parquet("data/jewels.parquet")
jewel_data = (
    all_jewels
    #
    .explode("jewel_talent_list").select(
        "name",
        "jewel_lvl",
        pl.col("jewel_talent_list").struct.field("name").alias("skill_name"),
        pl.col("jewel_talent_list").struct.field("lvl").alias("skill_lvl"),
    )
)
print(jewel_data.to_pandas().head(5).to_markdown(index=False))

| name                  |   jewel_lvl | skill_name           |   skill_lvl |
|:----------------------|------------:|:---------------------|------------:|
| Joyau attaque [1]     |           1 | Machine de guerre    |           1 |
| Joyau attaque II [2]  |           2 | Machine de guerre    |           2 |
| Joyau attaque III [3] |           3 | Machine de guerre    |           3 |
| Joyau vengeance [2]   |           2 | Vengeance            |           1 |
| Joyau riposte [3]     |           3 | Poussée d'adrénaline |           1 |


In [48]:
# Create regirstry entries to store
# the number of times we use each jewel.
# We separated the jewel types for later convenience
var_registry["jewel_usages"] = {jewel_type: {} for jewel_type in range(1, 4)}

# Register each single jewel
unique_jewel_names = jewel_data["name"].unique().sort().to_list()
for jewel_name in unique_jewel_names:
    jewel_datasheet = jewel_data.filter(pl.col("name") == jewel_name).to_dicts()
    jewel_level = jewel_datasheet[0]["jewel_lvl"]

    # Create a variable that register the number of jewel use
    var_nb_of_jewel_use = model.NewIntVar(
        lb=0,
        ub=100,
        name=f"nb_of_jewel_use_for_skill_{skill_name}_jewel_name_{jewel_name}",
    )
    # Add the number of jewel usages to the registry
    var_registry["jewel_usages"][jewel_level][jewel_name] = var_nb_of_jewel_use

    # A jewel can have multiple skills, so we iterate over each one
    for skill_data in jewel_datasheet:
        skill_name, skill_level = (skill_data[_] for _ in ["skill_name", "skill_lvl"])

        # Create a variable that is the total number of skill points inherited from
        # this jewel
        var_nb_of_skill_points_inherited = model.NewIntVar(
            lb=0,
            ub=100,
            name=f"nb_of_skill_points_inherited_for_skill_{skill_name}_jewel_name_{jewel_name}",
        )
        # Link those 2 using a hard constraint:
        # the number of skill points is the number of jewels
        # times the amount of skill points per jewel
        model.Add(var_nb_of_skill_points_inherited == var_nb_of_jewel_use * skill_level)

        if skill_name not in var_registry["skill_list"].keys():
            var_registry["skill_list"][skill_name] = []

        # Add the number of skill points to the registry
        var_registry["skill_list"][skill_name].append(var_nb_of_skill_points_inherited)

In [50]:
# Register the total amount of jewel uses for each
# jewel size
jewel_uses = {
    jewel_level: sum(var_registry["jewel_usages"][jewel_level].values())
    for jewel_level in range(1, 4)
}

# Create constraints to limit the number of jewel uses

# Jewels-3 can only go in emplacements of size 3
model.Add(jewel_uses[3] <= var_registry["jewel_emplacement_sums"][3])

# Jewels-2 can go in emplacements of size 2 and 3
model.Add(
    jewel_uses[2]
    <= var_registry["jewel_emplacement_sums"][2]
    + var_registry["jewel_emplacement_sums"][3]
    - jewel_uses[3]
)

# Jewels-1 can go in emplacements of size 1, 2, and 3
model.Add(
    jewel_uses[1]
    <= var_registry["jewel_emplacement_sums"][1]
    + var_registry["jewel_emplacement_sums"][2]
    + var_registry["jewel_emplacement_sums"][3]
    - jewel_uses[2]
    - jewel_uses[3]
)

In [53]:
# Compute the number of free jewel emplacements for each jewel type
nb_free_jewel_emplacements = {
    jewel_level: var_registry["jewel_emplacement_sums"][jewel_level]
    - jewel_uses[jewel_level]
    for jewel_level in range(1, 4)
}

# Unify the total amount of emplacements
# Prioritize the number of free jewel-3 emplacements first
# then free jewel-2 emplacements
# then free jewel-1 emplacements
objective_free_jewel_emplacements = sum(
    [nb_free_jewel_emplacements[jewel_level] * 10**jewel_level]
)

In [74]:
objective_function = sum(
    [
        skill_objective * 10**9,
        nb_free_jewel_emplacements * 10**3,
        total_amount_of_skill_points,
    ]
)

skill_sum_capped_for_skill_Géologiste(0..100)